In [2]:
!pip install pandas geopy
!pip install pulp

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ------------------- -------------------- 8.1/16.4 MB 42.1 MB/s eta 0:00:01
   ---------------------------------------  16.3/16.4 MB 44.5 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 39.7 MB/s  0:00:00


In [1]:
import pandas as pd
import numpy as np

# --- CONFIGURACIÓN DE COSTOS ASUMIDOS ---
# Para la Parte 1 (Matriz de Costos C_ij)
# Usamos un factor grande para que la diferencia de coordenadas se traduzca en un costo razonable.
COSTO_UNITARIO_POR_COORD_UNIT = 500.0 

# Para la Parte 2 (Parámetros Operacionales)
COSTO_REBALANCEO_POR_MINUTO = 0.15 # Costo lineal de mover una bici por minuto (€)
PENALIZACION_ALPHA = 5.0          # Factor para la función de coste no lineal de rechazo (€)

# --------------------------------------------------------------------------------------
# 1. LIMPIEZA Y PREPARACIÓN DE DATOS DE ESTACIONES (NODOS)
# --------------------------------------------------------------------------------------

# Cargar y limpiar el archivo de estaciones
df_stations = pd.read_csv('bikestationbicimad_csv.csv', delimiter=';')

# Limpiar coordenadas (reemplazar coma por punto y convertir a float)
df_stations['POINT_X'] = df_stations['POINT_X'].astype(str).str.replace(',', '.').astype(float)
df_stations['POINT_Y'] = df_stations['POINT_Y'].astype(str).str.replace(',', '.').astype(float)

# Limpiar ID de Estación y convertir a entero (Int64 para manejar errores de coerción)
df_stations['number'] = pd.to_numeric(df_stations['number'], errors='coerce')
df_stations = df_stations.dropna(subset=['number'])
df_stations['ID_Estacion'] = df_stations['number'].astype(int)

# DataFrame de Nodos (Tabla 1 & 2)
df_nodes = df_stations[['ID_Estacion', 'POINT_Y', 'POINT_X', 'TotalBases']].copy()
df_nodes.columns = ['ID_Estacion', 'Latitud', 'Longitud', 'Capacidad_Cj']
df_nodes = df_nodes.set_index('ID_Estacion')

# Obtener IDs válidos para el filtrado de viajes
valid_ids = df_nodes.index.unique().tolist()


# --------------------------------------------------------------------------------------
# 2. CREACIÓN DE LA MATRIZ DE DEMANDA (D_ij)
# --------------------------------------------------------------------------------------

print("--- Generando Matriz de Demanda (D_ij) ---")
df_trips = pd.read_csv('trips_23_01_January.csv', delimiter=';')

# Filtrar y limpiar IDs de estación en los viajes
df_trips = df_trips.dropna(subset=['station_unlock', 'station_lock'])
df_trips['station_unlock'] = df_trips['station_unlock'].astype('Int64')
df_trips['station_lock'] = df_trips['station_lock'].astype('Int64')

# Filtrar viajes solo entre estaciones válidas
df_trips = df_trips[df_trips['station_unlock'].isin(valid_ids) & 
                    df_trips['station_lock'].isin(valid_ids)]

# Agrupar y contar viajes
df_demanda = df_trips.groupby(['station_unlock', 'station_lock']).size().reset_index(name='Total_Viajes')

# Pivotear para crear la Matriz D_ij y rellenar celdas vacías (sin viajes) con 0
matriz_demanda_Dij = df_demanda.pivot(index='station_unlock', columns='station_lock', values='Total_Viajes').fillna(0)

# Guardar la matriz
matriz_demanda_Dij.to_csv('matriz_demanda_Dij.csv')
print("Matriz de Demanda (D_ij) guardada en 'matriz_demanda_Dij.csv'")


# --------------------------------------------------------------------------------------
# 3. CREACIÓN DE LA MATRIZ DE COSTOS (C_ij)
# --------------------------------------------------------------------------------------

print("\n--- Generando Matriz de Costos (C_ij) ---")

stations_list = df_nodes.index.tolist()
num_stations = len(stations_list)
cost_matrix = np.zeros((num_stations, num_stations))

latitudes = df_nodes['Latitud'].values
longitudes = df_nodes['Longitud'].values

# Cálculo de la distancia Euclídea (proxy de costo) entre todos los pares
for i in range(num_stations):
    lat_i = latitudes[i]
    lon_i = longitudes[i]
    
    # Vectorización para cálculo de distancia Euclídea
    diff_lat = lat_i - latitudes
    diff_lon = lon_i - longitudes
    dist_unit = np.sqrt(diff_lat**2 + diff_lon**2)
    
    # Calcular el costo unitario C_ij
    cost_matrix[i, :] = dist_unit * COSTO_UNITARIO_POR_COORD_UNIT

# Crear el DataFrame final
matriz_costos_Cij = pd.DataFrame(cost_matrix, index=stations_list, columns=stations_list)
matriz_costos_Cij = matriz_costos_Cij.round(3)

# Guardar la matriz
matriz_costos_Cij.to_csv('matriz_costos_Cij.csv')
print("Matriz de Costos (C_ij) guardada en 'matriz_costos_Cij.csv'")


# --------------------------------------------------------------------------------------
# 4. CREACIÓN DE LA TABLA DE PARÁMETROS OPERACIONALES (Parte 2)
# --------------------------------------------------------------------------------------

print("\n--- Generando Parámetros Operacionales (Parte 2) ---")

# Identificar la estación crítica (mayor actividad total)
departures = df_trips.groupby('station_unlock').size()
arrivals = df_trips.groupby('station_lock').size()
total_activity = departures.add(arrivals, fill_value=0)
critical_station_id = total_activity.idxmax()

# Preparar datos de tiempo
df_trips['unlock_date'] = pd.to_datetime(df_trips['unlock_date'])
df_trips['lock_date'] = pd.to_datetime(df_trips['lock_date'])
df_critical_trips = df_trips[(df_trips['station_unlock'] == critical_station_id) | 
                             (df_trips['station_lock'] == critical_station_id)].copy()
df_critical_trips['unlock_hour'] = df_critical_trips['unlock_date'].dt.hour
df_critical_trips['lock_hour'] = df_critical_trips['lock_date'].dt.hour

# Calcular flujos horarios y convertirlos a tasa por minuto
hourly_departures = df_critical_trips.groupby('unlock_hour').size().reindex(range(24), fill_value=0)
hourly_arrivals = df_critical_trips.groupby('lock_hour').size().reindex(range(24), fill_value=0)

minutes_in_period = 60 * 31 # 60 minutos * 31 días de enero
lambda_h = hourly_departures / minutes_in_period # Salidas (Demanda)
mu_h = hourly_arrivals / minutes_in_period     # Llegadas (Oferta)

# Crear la tabla final de parámetros
df_operaciones = pd.DataFrame({
    'ID_Periodo_h': range(24),
    'Flujo_Salida_por_Min': lambda_h.values, 
    'Flujo_Llegada_por_Min': mu_h.values,       
    'Coste_Rebalanceo_Cm': COSTO_REBALANCEO_POR_MINUTO,
    'Penalizacion_Cuadratica_Alpha': PENALIZACION_ALPHA,
    'Estacion_Critica_ID': critical_station_id
})

# Guardar la tabla
df_operaciones.to_csv('parametros_operacionales_bicimad.csv', index=False)
print("Parámetros Operacionales guardados en 'parametros_operacionales_bicimad.csv'")

--- Generando Matriz de Demanda (D_ij) ---
Matriz de Demanda (D_ij) guardada en 'matriz_demanda_Dij.csv'

--- Generando Matriz de Costos (C_ij) ---
Matriz de Costos (C_ij) guardada en 'matriz_costos_Cij.csv'

--- Generando Parámetros Operacionales (Parte 2) ---
Parámetros Operacionales guardados en 'parametros_operacionales_bicimad.csv'
